In [1]:
# Cody Giles - Student ID: 010506641
# C964 Capstone - Movie Audience Rating Predictor aka The MARP

import pandas as pd
from xgboost import XGBClassifier
# read data
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data['data'], data['target'], test_size=.2)
# create model instance
bst = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
# fit model
bst.fit(X_train, y_train)
# make predictions
preds = bst.predict(X_test)
bst.score(X_test, y_test)

1.0

In [2]:
# Bring in the dataset CSV
movies_df = pd.read_csv('imdb_movies.csv', encoding='utf-8')

# Breaks out the CSV into just the columns we're interested in
selected_columns = movies_df[['names', 'score', 'crew', 'budget_x']]

# Breaks out the cast / who's starring in the movies
starring_df = movies_df
starring_df['crew'] = starring_df['crew'].apply(lambda x: [y for y in str(x).split(', ')[::2]])
cast = starring_df[['crew']]

# SCORING THE ACTORS
# Define the multipliers (1st actor gets a 3x multiplier, 2nd gets 2x, 3rd gets 1.5x)
multipliers = [3, 2, 1.5, 1]

# Dictionary to store actor scores and the number of movies each actor has been in
actor_scores = {}
actor_movie_count = {}

for idx, row in movies_df.iterrows():
    actor_list = row['crew']
    user_score = row['score']  # Get the user score for the movie
    for i, actor in enumerate(actor_list):
        multiplier = multipliers[i] if i < len(multipliers) else 1  # Apply the multiplier
        weighted_score = multiplier * user_score  # Multiply by the user score
        # Update the actor's score
        if actor in actor_scores:
            actor_scores[actor] += weighted_score
            actor_movie_count[actor] += 1
        else:
            actor_scores[actor] = weighted_score
            actor_movie_count[actor] = 1

# for actor in actor_scores:
#    actor_scores[actor] /= actor_movie_count[actor]

final_actor_scores = {actor: actor_scores[actor] / actor_movie_count[actor] for actor in actor_scores}

# Convert the actor_scores dictionary to a DataFrame
actor_scores_df = pd.DataFrame(list(actor_scores.items()), columns=['crew', 'score'])

# Output actor_scores df
actor_scores_df


,crew,score
0,Michael B. Jordan,2336.0
1,Tessa Thompson,1212.5
2,Jonathan Majors,711.0
3,Wood Harris,446.0
4,Phylicia Rashad,702.0
...,...,...
39482,Jesse Collins,55.0
39483,Jack Langedijk,55.0
39484,Nina Herzog,210.0
39485,Gardner Jaas,70.0


In [2]:

x = df3.select_dtypes(['number']).drop('score', axis=1)
y = df3['score']

from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBRegressor

X_train, X_test, y_train, y_test = train_test_split(X,y)

model.score(X_test.to_numpy(), y_test.to_numpy())


,crew,score
0,Michael B. Jordan,2336.0
1,Tessa Thompson,1212.5
2,Jonathan Majors,711.0
3,Wood Harris,446.0
4,Phylicia Rashād,702.0
...,...,...
39486,Jesse Collins,55.0
39487,Jack Langedijk,55.0
39488,Nina Herzog,210.0
39489,Gardner Jaas,70.0
